<div style="float:left">
    <h1 style="width:600px">Assessment</h1>
    <h3 style="width:600px">CASA0002: Urban Simulation</h3>
    <h3 style="width:600px">Author: Andres Restrepo</h3>

</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

## Required libraries

In [ ]:
import timeit
start_time = timeit.default_timer()
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Data reading

### Underground network

In [ ]:
G = nx.read_graphml('Data/Inputs/london.graph')

In [ ]:
type(G)

### OD matrix

In [ ]:
OD = pd.read_csv('Data/Inputs/OD_matrix.csv',index_col='station_origin')
OD.rename_axis('station_destination', axis=1,inplace=True)

In [ ]:
type(OD)

### Flow data

In [ ]:
london_OD_AMpeak = pd.read_csv('Data/Inputs/london_flows.csv')

In [ ]:
type(london_OD_AMpeak)

## Data inspection

### Underground network

In [ ]:
print(nx.info(G))

In [ ]:
list(G.nodes(data = True))[0]

In [ ]:
# #check that all distances are greater or equal to euclidean distance
# for u,v, data in G.edges(data=True):
#     assert(data['length'] >= distance.euclidean(pos[u], pos[v]))

In [ ]:
#if want to know what is the maximum distance between stations
max(dict(G.edges).items(), key=lambda x: x[1]['length'])

In [ ]:
#if want to know to which stations Baker Street is directly connected to
Baker_Street = [(u,v) for  u,v in G.edges() if u == 'Baker Street' or v == 'Baker Street']
Baker_Street

In [ ]:
#if want to know to which stations Baker Street is directly connected to
Stratford = [(u,v) for  u,v in G.edges() if u == 'Stratford' or v == 'Stratford']
Stratford

In [ ]:
#We can also check the degree of the nodes in our network and check that they make sense
deg_london = nx.degree(G)

In [ ]:
df = pd.DataFrame(index = dict(deg_london).keys())
df['degree'] = dict(deg_london).values()

In [ ]:
df.sort_values('degree', ascending =False).head(20)

The inspecting results are consistent with the results of Practical 10.

### OD matrix

In [ ]:
OD.info()

In [ ]:
OD.head()

The inspecting results are consistent with the results of Practical 10.

### Flow data

In [ ]:
london_OD_AMpeak.info()

In [ ]:
london_OD_AMpeak.head()

The inspecting results are consistent with the results of Practical 10.

# London’s underground resilience

## Topological network

![img](https://tfl.gov.uk/cdn/static/cms/images/london-rail-and-tube-services-map.gif)

In [ ]:
#since coords tuples are stored as string, need to convert them back to tuples using eval()
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])

In [ ]:
list(G.nodes(data = True))[0]

In [ ]:
# We can plot the tube network with the names of the stations as labels
fig, ax = plt.subplots(figsize=(25,20))

pos = nx.get_node_attributes(G, 'coords')

# Nodes
nx.draw_networkx_nodes(G,pos,node_size=50,node_color='b')
# Edges
nx.draw_networkx_edges(G,pos,arrows=False,width=0.2)
# Labels
nx.draw_networkx_labels(G,pos, font_size=10, font_color='black')

plt.title("London underground network",fontsize=20)
plt.axis("off")
plt.show()

**It better to normalized to compare more easily?**

### Centrality measures

**Include eigen vector centrality?**

#### Calculation

In [ ]:
df_centrality = pd.DataFrame(index=G.nodes())

deg_london =nx.degree_centrality(G)
nx.set_node_attributes(G,dict(deg_london),'degree')
df_centrality['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))

clos_london = nx.closeness_centrality(G)
nx.set_node_attributes(G,clos_london,'closeness_t')
df_centrality['closeness_t'] = pd.Series(nx.get_node_attributes(G, 'closeness_t'))

bet_london = nx.betweenness_centrality(G)
nx.set_node_attributes(G,bet_london,'betweenness_t')
df_centrality['betweenness_t'] = pd.Series(nx.get_node_attributes(G, 'betweenness_t'))

eig_london = nx.eigenvector_centrality_numpy(G)
nx.set_node_attributes(G,bet_london,'eigenvector_t')
df_centrality['eigenvector_t'] = pd.Series(nx.get_node_attributes(G, 'eigenvector_t'))
# Change the tolerance value



In [ ]:
df_centrality.head(10).sort_values(by=['degree'],ascending=False)

#### Degree centrality

In [ ]:
df_centrality['degree'].sort_values(ascending=False)

In [ ]:
# Plot of degree centrality
degree_values=[(i[1]['degree']) for i in G.nodes(data=True)]
# Sclaling using the maximum value of the degree
deg_color=[(i[1]['degree']/(max(degree_values))) for i in G.nodes(data=True)]
# Sclaling using the maximum value of the degree and multipling it by 50
deg_size=[(i[1]['degree']/(max(degree_values)))*100 for i in G.nodes(data=True)]


# Plot graph
pos=pos
fig, ax = plt.subplots(figsize=(12,12))
# Edges
nx.draw_networkx_edges(G, pos,edge_color='gray', 
        width=0.4)
# Nodes
nod=nx.draw_networkx_nodes(G,
        pos = pos,
        node_color= deg_color,
        node_size= deg_size)

plt.colorbar(nod,label="Degree centrality",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London underground degree centrality",fontsize=15)
plt.show()

#### Closeness centrality

In [ ]:
df_centrality['closeness_t'].sort_values(ascending=False)[0:10]

In [ ]:
# Lets set color and width of nodes according to the closeness values
close_t_values=[(i[1]['closeness_t']) for i in G.nodes(data=True)]

# close_t_color=[(i[1]['closeness_t']-min(close_t_values))/(max(close_t_values)-min(close_t_values)) for i in G.nodes(data=True)]
# close_t_size=[((i[1]['closeness_t']-min(close_t_values))/(max(close_t_values)-min(close_t_values))*50) for i in G.nodes(data=True)]

close_t_color=[(i[1]['closeness_t']/max(close_t_values)) for i in G.nodes(data=True)]
close_t_size=[(i[1]['closeness_t']/max(close_t_values)*100) for i in G.nodes(data=True)]

# Plot graph
fig, ax = plt.subplots(figsize=(12,12))

nx.draw_networkx_edges(G, pos,edge_color='gray', 
        width=0.4)

nod=nx.draw_networkx_nodes(G,
        pos = pos,
        node_color= close_t_color,
        node_size= close_t_size)

plt.colorbar(nod,label="Topological closeness centrality",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London underground topological closeness centrality",fontsize=15)
plt.show()

#### Betweenness centrality

In [ ]:
df_centrality['betweenness_t'].sort_values(ascending=False)[0:10]

In [ ]:
# Lets set colour and size of nodes according to betweenness values
bet_t_values=[(i[1]['betweenness_t']) for i in G.nodes(data=True)]

bet_t_color=[(i[1]['betweenness_t']/max(bet_t_values)) for i in G.nodes(data=True)]
bet_t_size=[(i[1]['betweenness_t']/max(bet_t_values))*100 for i in G.nodes(data=True)]

# Plot graph
fig, ax = plt.subplots(figsize=(12,12))

nx.draw_networkx_edges(G, pos,edge_color='gray', width=0.4)

nod=nx.draw_networkx_nodes(G, pos = pos, node_color= bet_t_color, node_size= bet_t_size)

plt.colorbar(nod,label="Topological betweenness centrality",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London underground topological betweenness centrality",fontsize=15)
plt.show()

#### Eigenvector  centrality

In [ ]:
df_centrality['eigenvector_t'].sort_values(ascending=False)[0:10]

In [ ]:
# Eigenvetor centrality
eig_t_values=[(i[1]['eigenvector_t']) for i in G.nodes(data=True)]
eig_t_color=[(i[1]['eigenvector_t']/max(eig_t_values)) for i in G.nodes(data=True)]
eig_t_size=[(i[1]['eigenvector_t']/max(eig_t_values))*100 for i in G.nodes(data=True)]

# Plot graph
fig, ax = plt.subplots(figsize=(12,12))

nx.draw_networkx_edges(G, pos,edge_color='gray', width=0.4)

nod=nx.draw_networkx_nodes(G, pos = pos, node_color= eig_t_color, node_size= eig_t_size)

plt.colorbar(nod,label="Topological eigenvector centrality",orientation="horizontal", shrink=0.5)
plt.axis("off")
plt.title("London underground topological eigenvector centrality",fontsize=15)
plt.show()

#### Results summary

In [ ]:
df_ranking = pd.DataFrame()
rank = list(range(1, df_centrality.shape[0]+1))
df_ranking['rank'] = rank

degree_rank = df_centrality.sort_values(by='degree',ascending=False).index.values.tolist()
degree_value = df_centrality['degree'].sort_values(ascending=False).values.tolist()
clos_rank = df_centrality.sort_values(by='closeness_t',ascending=False).index.values.tolist()
clos_value = df_centrality['closeness_t'].sort_values(ascending=False).values.tolist()
bet_rank = df_centrality.sort_values(by='betweenness_t',ascending=False).index.values.tolist()
bet_value = df_centrality['betweenness_t'].sort_values(ascending=False).values.tolist()
eig_rank = df_centrality.sort_values(by='eigenvector_t',ascending=False).index.values.tolist()
eig_value = df_centrality['eigenvector_t'].sort_values(ascending=False).values.tolist()


df_ranking['Degree'] = degree_rank
df_ranking['Degree value'] = degree_value
df_ranking['Topological closeness'] = clos_rank
df_ranking['Closeness value'] = clos_value
df_ranking['Topological betweenness'] = bet_rank
df_ranking['Betweenness value'] = bet_value
df_ranking['Topological Eigenvector'] = eig_rank
df_ranking['Eigenvector value'] = eig_value

df_ranking.head(10)


In [ ]:
# df_ranking.loc[]

### Impact measures

**Nodes**

In [ ]:
#list(G.nodes())

**Number of nodes**

In [ ]:
G.number_of_nodes()

**Diameter**

In [ ]:
d=nx.diameter(G)
print(d)

**Average shortest path**

In [ ]:
asp = nx.average_shortest_path_length(G)
print(asp)

**Number of connected components**

In [ ]:
nx.number_connected_components(G)

**Size of the largest component**

In [ ]:
# To obtain the largest component as a new graph: 

# Get the list of components:
components = nx.connected_components(G)
# Use the max() command to find the largest one:
largest_component = max(components, key=len)
# Create a "subgraph" of the largest component
Largest_subgraph = G.subgraph(largest_component)

In [ ]:
Largest_subgraph.number_of_nodes()

In [ ]:
type(components)

In [ ]:
# You can compute the diameter of the largest component
d_lc = nx.diameter(Largest_subgraph)
d_lc

**Size of multiple components**

In [ ]:
# To comput the size ( number of node) of each component
Subg = [G.subgraph(c).copy() for c in nx.connected_components(G)]
[len(subgraph.nodes) for subgraph in Subg]

In [ ]:
list([len(subgraph.nodes) for subgraph in Subg])

In [ ]:
type([len(subgraph.nodes) for subgraph in Subg])

### Node removal

#### Non-sequential

In [ ]:
# Copy of the initial network
G_copy = G.copy()

In [ ]:
# Create a blank df
non_sq_df = pd.DataFrame()
#Create empy columns
non_sq_df['Nodes'] = np.nan
non_sq_df['Removed nodes'] = np.nan
non_sq_df['Diameter'] = np.nan
non_sq_df['Average shortest path'] = np.nan
non_sq_df['Number of connected components'] = np.nan
non_sq_df['Size of largest connected component'] = np.nan

In [ ]:
# List of ranking lists
ranks = [degree_rank, clos_rank, bet_rank, eig_rank]

In [ ]:
labels = ['Degree', 'Closeness', 'Betweenness', 'Eigenvector']

In [ ]:
# For loop to calculate impact measures in non-sequential node removal
counter = 0
for r in ranks:
    #print(r)
    G_i = G_copy.copy()

    for station in r[0:21]: 
        # Remove a node from the graph
        
        #print(station) 
        #G_i.remove_node(station) # Erasing before the first impact measurement is performed

        # Check if the graph is connected
        if nx.is_connected(G_i):
            # Calculate the number of nodes in the graph
            nodes = nx.number_of_nodes(G_i)
            # Number of removed nodes
            re_nodes = G_copy.number_of_nodes() - G_i.number_of_nodes()
            # Diameter
            diameter = nx.diameter(G_i)
            #Average shortest path of the connected component
            avr_shortest = nx.average_shortest_path_length(G_i)
            # Number of connected componets
            num_con = nx.number_connected_components(G_i)
            # Size of the largest component
            components = nx.connected_components(G_i)
            # Use the max() command to find the largest one:
            largest_component = max(components, key=len)
            # Create a "subgraph" of the largest component
            Largest_subgraph = G.subgraph(largest_component)
            size_lar_com = Largest_subgraph.number_of_nodes()

            # Add the station, number of nodes, and diameter to the DataFrame
            # non_sq = non_sq.append({'Nodes': nodes, 'Removed nodes': re_nodes, 'Diameter for largest component': diameter,'Average shortest path':avr_shortest}, ignore_index=True)
            non_sq_df = pd.concat([non_sq_df, pd.DataFrame({'Nodes': [nodes], 'Removed nodes': [re_nodes], 'Diameter': [diameter],'Average shortest path':[avr_shortest], 'Centrality measure':labels[counter], 'Number of connected components': num_con, 'Size of largest connected component':size_lar_com})], ignore_index=True)
            G_i.remove_node(station)
        else:
            # Find the largest connected component
            lar_component = max(nx.connected_components(G_i), key=len)
            # Create a subgraph of the largest connected component
            Largest_subgraph = G_i.subgraph(lar_component)
            # Calculate the number of nodes in the subgraph
            nodes = nx.number_of_nodes(G_i)
            # Number of removed nodes
            re_nodes = G_copy.number_of_nodes() - G_i.number_of_nodes()
            # Calculate the diameter of the largest connedted component
            diameter = nx.diameter(Largest_subgraph)
            # Average shortest path of the largest connedted component
            avr_shortest = nx.average_shortest_path_length(Largest_subgraph)
            # Number of connected componets
            num_con = nx.number_connected_components(G_i)
            # Size of the largest component
            size_lar_com = Largest_subgraph.number_of_nodes()
           

            # Add the station, number of nodes, and diameter to the DataFrame
            non_sq_df = pd.concat([non_sq_df, pd.DataFrame({'Nodes': [nodes], 'Removed nodes': [re_nodes], 'Diameter': [diameter],'Average shortest path':[avr_shortest], 'Centrality measure':labels[counter], 'Number of connected components': num_con, 'Size of largest connected component':size_lar_com})], ignore_index=True)
            G_i.remove_node(station)
        
    counter += 1
        
    #print(counter)
        

In [ ]:
non_sq_df.head()

In [ ]:
# #Working code
# fig, [ax1,ax2,ax3,ax4] = plt.subplots(1, 4, sharey=False, tight_layout=True,figsize=(20, 7))
# plt.suptitle('Impact measure - Non-sequential node removal',fontsize=22)
# ax1.set_ylabel("Impact measurement value",fontsize=14)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Diameter",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure',linewidth = 2.5)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Average shortest path",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=ax3,style='Centrality measure',linewidth = 2.5)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=ax4,style='Centrality measure',linewidth = 2.5)


# ax1.set_title("Diameter",fontsize=16)
# ax2.set_title("Average shortest path",fontsize=16)
# ax3.set_title("Number of connected components",fontsize=16)
# ax4.set_title("Size of largest connected component",fontsize=16)

# ax2.set_ylabel('')
# ax3.set_ylabel('')
# ax4.set_ylabel('')

# # ax1.set_yticklabels([])
# # ax2.set_yticklabels([]) # remove lable axis
# # ax2.set_yticks([])


# ax1.get_legend().remove()
# ax2.get_legend().remove()
# ax3.get_legend().remove()
# plt.legend(bbox_to_anchor=(1.7, 0.5),loc='center right',title='Centrality\nmeasure',fontsize=14,title_fontsize=16)

# ax1.set_xlabel("Removed nodes",fontsize=14)
# ax2.set_xlabel("Removed nodes",fontsize=14)
# ax3.set_xlabel("Removed nodes",fontsize=14)
# ax4.set_xlabel("Removed nodes",fontsize=14)

# for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
# 	label.set_fontsize(14)

# for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
# 	label.set_fontsize(14)
    
# for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
# 	label.set_fontsize(14)
    
# for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
# 	label.set_fontsize(14)
    
# # plt.savefig("Dis_line_ratio_meta_M1_M2_M3.jpg",dpi=300)

In [ ]:
# Selecting data for plotting
non_sq_df[(non_sq_df['Centrality measure']=='Degree') | (non_sq_df['Centrality measure']=='Betweenness') | (non_sq_df['Centrality measure']=='Eigenvector')].tail()

In [ ]:
# Selecting data for plotting
non_sq_df[(non_sq_df['Centrality measure']=='Degree') | (non_sq_df['Centrality measure']=='Betweenness') | (non_sq_df['Centrality measure']=='Eigenvector')].head()

In [ ]:
# Subsetting the df for plotting
non_sq_df_plot = non_sq_df[(non_sq_df['Centrality measure']=='Degree') | (non_sq_df['Centrality measure']=='Betweenness') | (non_sq_df['Centrality measure']=='Eigenvector')]

In [ ]:
# Setting the theme
sns.set_theme()

In [ ]:
#Plotting just 3 centrality measures
fig, [ax1,ax2] = plt.subplots(1, 2, sharey=False, tight_layout=True,figsize=(20, 7))
plt.suptitle('Impact measure - Non-sequential node removal',fontsize=22)
ax1.set_ylabel("Impact measurement value",fontsize=14)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Diameter",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure',linewidth = 2.5)
# sns.lineplot(data=non_sq_df, x="Removed nodes", y="Average shortest path",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=non_sq_df_plot, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=non_sq_df_plot, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)


# ax1.set_title("Diameter",fontsize=16)
# ax2.set_title("Average shortest path",fontsize=16)
ax1.set_title("Number of connected components",fontsize=16)
ax2.set_title("Size of largest connected component",fontsize=16)

ax1.set_ylabel('')
ax2.set_ylabel('')
# ax4.set_ylabel('')

# ax1.set_yticklabels([])
# ax2.set_yticklabels([]) # remove lable axis
# ax2.set_yticks([])


ax1.get_legend().remove()
# ax2.get_legend().remove()
# ax3.get_legend().remove()
plt.legend(bbox_to_anchor=(1.3, 0.5),loc='center right',title='Centrality\nmeasure',fontsize=14,title_fontsize=16)

ax1.set_xlabel("Removed nodes",fontsize=14)
ax2.set_xlabel("Removed nodes",fontsize=14)
# ax3.set_xlabel("Removed nodes",fontsize=14)
# ax4.set_xlabel("Removed nodes",fontsize=14)

for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
	label.set_fontsize(14)

for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
	label.set_fontsize(14)
    
# for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
# 	label.set_fontsize(14)
    
# for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
# 	label.set_fontsize(14)
    
# plt.savefig("Dis_line_ratio_meta_M1_M2_M3.jpg",dpi=300)

#### Sequential

In [ ]:
measures = ['degree','closeness','betweenness','eigenvector']

In [ ]:
# Define a dictionary of functions
functions_dict = {'degree': nx.degree_centrality,
                  'closeness': nx.closeness_centrality,
                  'betweenness': nx.betweenness_centrality,
                 'eigenvector':nx.eigenvector_centrality_numpy}

In [ ]:
# Create a blank df
sq_df = pd.DataFrame()
#Create empy columns
sq_df['Nodes'] = np.nan
sq_df['Removed nodes'] = np.nan
sq_df['Diameter'] = np.nan
sq_df['Average shortest path'] = np.nan
sq_df['Number of connected components'] = np.nan
sq_df['Size of largest connected component'] = np.nan
sq_df['Node removed'] = np.nan

In [ ]:
# Working code
counter = 0
for m in measures:
    #print(r)
    G_i = G_copy.copy()
    # print(m)

    for i in range(21): 
        # Remove a node from the graph
        
        # Calculate the centrality measure
        centrality_list = functions_dict[m](G_i)
        max_centrality_node = max(centrality_list, key=centrality_list.get)
        # print(max_centrality_node)
        
        # Check if the graph is connected
        if nx.is_connected(G_i):
            # Calculate the number of nodes in the graph
            nodes = nx.number_of_nodes(G_i)
            # Number of removed nodes
            re_nodes = G_copy.number_of_nodes() - G_i.number_of_nodes()
            # Diameter
            diameter = nx.diameter(G_i)
            #Average shortest path of the connected component
            avr_shortest = nx.average_shortest_path_length(G_i)
            # Number of connected componets
            num_con = nx.number_connected_components(G_i)
            # Size of the largest component
            components = nx.connected_components(G_i)
            # Use the max() command to find the largest one:
            largest_component = max(components, key=len)
            # Create a "subgraph" of the largest component
            Largest_subgraph = G.subgraph(largest_component)
            size_lar_com = Largest_subgraph.number_of_nodes()
        

            # Add the station, number of nodes, and diameter to the DataFrame
            # non_sq = non_sq.append({'Nodes': nodes, 'Removed nodes': re_nodes, 'Diameter for largest component': diameter,'Average shortest path':avr_shortest}, ignore_index=True)
            sq_df = pd.concat([sq_df, pd.DataFrame({'Nodes': [nodes], 'Removed nodes': [re_nodes], 'Diameter': [diameter],'Average shortest path':[avr_shortest], 'Centrality measure':labels[counter], 'Number of connected components': num_con, 'Size of largest connected component':size_lar_com,'Node removed':max_centrality_node})], ignore_index=True)
            G_i.remove_node(max_centrality_node)
        else:
            # Find the largest connected component
            lar_component = max(nx.connected_components(G_i), key=len)
            # Create a subgraph of the largest connected component
            Largest_subgraph = G_i.subgraph(lar_component)
            # Calculate the number of nodes in the subgraph
            nodes = nx.number_of_nodes(G_i)
            # Number of removed nodes
            re_nodes = G_copy.number_of_nodes() - G_i.number_of_nodes()
            # Calculate the diameter of the largest connedted component
            diameter = nx.diameter(Largest_subgraph)
            # Average shortest path of the largest connedted component
            avr_shortest = nx.average_shortest_path_length(Largest_subgraph)
            # Number of connected componets
            num_con = nx.number_connected_components(G_i)
            # Size of the largest component
            size_lar_com = Largest_subgraph.number_of_nodes()
           

            # Add the station, number of nodes, and diameter to the DataFrame
            sq_df = pd.concat([sq_df, pd.DataFrame({'Nodes': [nodes], 'Removed nodes': [re_nodes], 'Diameter': [diameter],'Average shortest path':[avr_shortest], 'Centrality measure':labels[counter], 'Number of connected components': num_con, 'Size of largest connected component':size_lar_com,'Node removed':max_centrality_node})], ignore_index=True)
            G_i.remove_node(max_centrality_node)
        
    counter += 1
        
    #print(counter)
        

In [ ]:
sq_df.head()

In [ ]:
# Working code
# Plot for sequential
fig, [ax1,ax2,ax3,ax4] = plt.subplots(1, 4, sharey=False, tight_layout=True,figsize=(20, 7))
plt.suptitle('Impact measure - Sequential node removal',fontsize=22)
ax1.set_ylabel("Impact measurement value",fontsize=14)
sns.lineplot(data=sq_df, x="Removed nodes", y="Diameter",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure', linewidth = 2.5)
sns.lineplot(data=sq_df, x="Removed nodes", y="Average shortest path",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=ax3,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=ax4,style='Centrality measure',linewidth = 2.5)


ax1.set_title("Diameter",fontsize=16)
ax2.set_title("Average shortest path",fontsize=16)
ax3.set_title("Number of connected components",fontsize=16)
ax4.set_title("Size of largest connected component",fontsize=16)

ax2.set_ylabel('')
ax3.set_ylabel('')
ax4.set_ylabel('')

# ax1.set_yticklabels([])
# ax2.set_yticklabels([]) # remove lable axis
# ax2.set_yticks([])


ax1.get_legend().remove()
ax2.get_legend().remove()
ax3.get_legend().remove()
plt.legend(bbox_to_anchor=(1.7, 0.5),loc='center right',title='Centrality\nmeasure',fontsize=14,title_fontsize=16)

ax1.set_xlabel("Removed nodes",fontsize=14)
ax2.set_xlabel("Removed nodes",fontsize=14)
ax3.set_xlabel("Removed nodes",fontsize=14)
ax4.set_xlabel("Removed nodes",fontsize=14)

for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
	label.set_fontsize(14)

for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
	label.set_fontsize(14)
    
for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
	label.set_fontsize(14)
    
for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
	label.set_fontsize(14)
    
# plt.savefig("Dis_line_ratio_meta_M1_M2_M3.jpg",dpi=300)

In [ ]:
# Selecting data for plotting
sq_df[(sq_df['Centrality measure']=='Degree') | (sq_df['Centrality measure']=='Betweenness') | (sq_df['Centrality measure']=='Eigenvector')].tail()

In [ ]:
# Selecting data for plotting
sq_df[(sq_df['Centrality measure']=='Degree') | (sq_df['Centrality measure']=='Betweenness') | (sq_df['Centrality measure']=='Eigenvector')].head()

In [ ]:
# Subsetting the df for plotting
sq_df_plot = sq_df[(sq_df['Centrality measure']=='Degree') | (sq_df['Centrality measure']=='Betweenness') | (sq_df['Centrality measure']=='Eigenvector')]

In [ ]:
# Testing code
# Plot for sequential for degre, betweenness and eigenvector
fig, [ax1,ax2] = plt.subplots(1, 2, sharey=False, tight_layout=True,figsize=(20, 7))
plt.suptitle('Impact measure - Sequential node removal',fontsize=22)
ax1.set_ylabel("Impact measurement value",fontsize=14)
# sns.lineplot(data=sq_df, x="Removed nodes", y="Diameter",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure', linewidth = 2.5)
# sns.lineplot(data=sq_df, x="Removed nodes", y="Average shortest path",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df_plot, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=ax1,style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df_plot, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=ax2,style='Centrality measure',linewidth = 2.5)


# ax1.set_title("Diameter",fontsize=16)
# ax2.set_title("Average shortest path",fontsize=16)
ax1.set_title("Number of connected components",fontsize=16)
ax2.set_title("Size of largest connected component",fontsize=16)

ax2.set_ylabel('')
# ax3.set_ylabel('')
# ax4.set_ylabel('')

# ax1.set_yticklabels([])
# ax2.set_yticklabels([]) # remove lable axis
# ax2.set_yticks([])


ax1.get_legend().remove()
# ax2.get_legend().remove()
# ax3.get_legend().remove()
plt.legend(bbox_to_anchor=(1.3, 0.5),loc='center right',title='Centrality\nmeasure',fontsize=14,title_fontsize=16)

ax1.set_xlabel("Removed nodes",fontsize=14)
ax2.set_xlabel("Removed nodes",fontsize=14)
# ax3.set_xlabel("Removed nodes",fontsize=14)
# ax4.set_xlabel("Removed nodes",fontsize=14)

for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
	label.set_fontsize(14)

for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
	label.set_fontsize(14)
    
# for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
# 	label.set_fontsize(14)
    
# for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
# 	label.set_fontsize(14)
    
# plt.savefig("Dis_line_ratio_meta_M1_M2_M3.jpg",dpi=300)

**Total results**

In [ ]:
plot_labels = ['Degree','Betweenness', 'Eigenvector']

In [ ]:
# Plot total results
fig, axes = plt.subplots(2, 2, sharey=False, tight_layout=True,figsize=(14, 8))
plt.suptitle('Impact measure results - Node removal',fontsize=22)
# axes[0,0].set_ylabel("Non-sequential",fontsize=14)
# axes[0,1].set_ylabel("Sequential",fontsize=14)

axes[0,0].set_title("Number of connected components",fontsize=16)
axes[0,1].set_title("Size of largest connected component",fontsize=16)


sns.lineplot(data=non_sq_df_plot, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=axes[0,0],style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=non_sq_df_plot, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=axes[0,1],style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df_plot, x="Removed nodes", y="Number of connected components",palette="tab10",hue='Centrality measure',ax=axes[1,0],style='Centrality measure',linewidth = 2.5)
sns.lineplot(data=sq_df_plot, x="Removed nodes", y="Size of largest connected component",palette="tab10",hue='Centrality measure',ax=axes[1,1],style='Centrality measure',linewidth = 2.5)

axes[0,0].set_ylabel('Non-sequential')
axes[0,1].set_ylabel('')
axes[1,0].set_ylabel('Sequential')
axes[1,1].set_ylabel('')

# axes[0,0].get_xaxis().set_visible(False)
# axes[0,1].get_xaxis().set_visible(False)

axes[0,0].get_legend().remove()
axes[0,1].get_legend().remove()
axes[1,0].get_legend().remove()
axes[1,1].get_legend().remove()
axes[0,0].set_xlabel('')
axes[0,1].set_xlabel('')

# Change the location of legend
# plt.legend(bbox_to_anchor=(1.45,1),loc='center right',title='Centrality\nmeasure',fontsize=14,title_fontsize=16)
fig.legend(bbox_to_anchor=(1.127,0.5),labels=plot_labels,loc='center right',title='Centrality\nmeasure',fontsize=12,title_fontsize=14)
# fig.legend(bbox_to_anchor=(0.5,-0.09),labels=plot_labels,loc="lower center",ncol=3,title='Centrality measure',fontsize=12,title_fontsize=14)
plt.savefig('Results_Impact_Measures_Total.jpg',dpi=300,bbox_inches='tight')

## Flows: weighted network

In [ ]:
# Copy of the original network
G_w = G_copy.copy()

In [ ]:
list(G_w.edges(data = True))[0]

In [ ]:
# Inspecting for zero valuea on the flows
zero_flows = [(u, v) for u, v, d in G_w.edges(data=True) if d['flows'] == 0]
print(zero_flows)

**Remove edge and node?**

In [ ]:
# Removing Battersea Park station as instructed by Elsa Arcuate
# Before removal
print(nx.info(G_w))

In [ ]:
# Removing node
# G_w.remove_node('Battersea Park')

In [ ]:
# After removal
print(nx.info(G_w))

In [ ]:
# Reviewing the removed node
# set(list(G_copy.nodes())) - set(list(G_w.nodes()))

In [ ]:
# # Invert the flows as it is supposed to be a distance
# inv_flows={(e1, e2):round(1./flows,7) for e1, e2, flows in G_w.edges(data='flows')}

# # Let us add the inverted weight as an attribute to the edges in the graph
# nx.set_edge_attributes(G_w, inv_flows, 'inv_flows')

# # Print head of edge list with attributes
# list(G_w.edges(data = True))[0:10]

# Spatial interaction models

### Models and calibration

#### Introduction

#### Parameter calibration

### Scenarios

#### Scenario A

#### Scenario B

#### Scenarios discussion

In [ ]:
print('Complete run time: ' + str((timeit.default_timer() - start_time)/60)+' minutes.')